# Lesson 2: Classes in the Wild

> _Disclaimer: These points were intended to be applied to Python, your mileage may vary._

- Overutilising or underutilising classes can lead to ruin
- Classes can be a powerful tool or an endless garden path

## Pros

- Can keep track of state
  - No need to pass parameters back and forth
  - No thread-unsafe global variables
  - Can logically initialise state and then use it
- Can organise a hierarcy of states that belong together
- Provide dot-methods for accessing properties
  - "ask, don't tell"
  
## Cons

- Can make code convoluted and hard to read
  - code spread across multiple files
  - logic for a single operation spread across multiple different parts

## Functions vs Classes

### Functions vs Methods

In python

- a **function** takes parameters, returns a value
- a **method** can be called on an object, and can access state in the object

## Baby steps

In [1]:
CONFIG = {
    'thing': 'a',
    'identifiier': 'b',
    'name': 'c'
}

_This isn't very safe if something goes wrong, and the IDE can't offer us any help_

In [2]:
CONFIG['identifier']

KeyError: 'identifier'

## Namedtuple

`namedtuple` == Quick 'n' dirty class!

Use when you just need to
- make sure that the correct keys/values are present
- access something a few times (safely) via a dot method rather than a dict key lookup

In [3]:
from collections import namedtuple

Config = namedtuple('config', ['thing', 'identifier', 'name'])

CONFIG = Config('a', 'b', 'c')

print(CONFIG)
print(CONFIG.thing, CONFIG.identifier)

config(thing='a', identifier='b', name='c')
a b


### A multi-level config object

In [4]:
from collections import namedtuple

# Define what your config objects need to contain
Endpoint = namedtuple('endpoints', ['key', 'url', 'timeout', 'n_workers'])
Endpoints = namedtuple('endpoints', ['customers', 'products'])

# Initialise all configs with their values
# Could read this from a JSON file, command-line args, or define it here
# Either way, the namedtuple will ensure that the result is the same
ENDPOINTS = {
    'customers': Endpoint('customers', 'customers/search/', 200, 2),
    'products': Endpoint('products', 'products/all/search/', 100, 2)
}

for endpoint, config in ENDPOINTS.items():
    print(endpoint, config.url)

customers customers/search/
products products/all/search/


Now let's try the failing example again

In [5]:
Config(**{
    'thing': 'a',
    'identifiier': 'b',
    'name': 'c'
})

TypeError: <lambda>() got an unexpected keyword argument 'identifiier'

Much better!
This means that we catch the error when Config is _**initialised**_, rather than when trying to _**access**_ 'identifier' later on.

This is also useful when loading a JSON config, and you need to make sure all the key are present

In [6]:
import json
raw = '{"identifier": 123, "name": "me", "thing": 123}'

Config(**json.loads(raw))

config(thing=123, identifier=123, name='me')

In [7]:
raw = '{"identifier": 123, "name": "me", "thing": 123, "extra": 1}'

Config(**json.loads(raw))

TypeError: <lambda>() got an unexpected keyword argument 'extra'

## Dataclasses

- Python 3.7+
- Syntactic sugar for defining an `__init__` method and instance variables
- also provides a nice `__repr__` method, and some other things

A regular class:

In [8]:
class Obj:
    def __init__(self, a=1, b=2, c='default'):
        self.a = a
        self.b = b
        self.c = c

Obj(1)

The same, but as a dataclass

In [9]:
from dataclasses import dataclass

@dataclass
class Obj:
    a: int = 1
    b: int = 2
    c: str = 'default'

Obj(1)

Obj(a=1, b=2, c='default')

In [10]:
Obj(d=5)

TypeError: __init__() got an unexpected keyword argument 'd'

---

## An Example

- You have a collection of items, in this case ids and emails
- Need to iterate through them, collect some values, and pass them on

In [11]:
from api import API

In [12]:
for i, el in enumerate(API.get('customers')):
    print(el)
    if i >= 12:
        break

7966
438
1237
7644
6843
None
None
None
None
None
None
None
None


For this exercise, we must consume a list of endpoints via key, and send them to their own file

In [13]:
for endpoint in ['customer', 'transactions']:
    if endpoint == 'customer':
        for i in API.get(endpoint+'s'):
            if i is None:
                break
            print(endpoint, next(API.get(endpoint, {'cid': i})))
    elif endpoint == 'transactions':
        for i in API.get(endpoint, {'ts': 0, 'te': 5}):
            if i is None:
                break
            print(endpoint, i)

customer carol32@gmail.com
customer brianmcfarland@yahoo.com
customer jamieestes@chan-miller.com
customer ugreen@parker.com
customer browncourtney@moore.com
transactions bbf47b52-34c0-4c55-94f9-4741abaa18c3
transactions ed93880a-f0f2-406a-a916-ab3fab358bdd
transactions c51cdcc8-5cdd-4635-b5cd-f012a3adc882
transactions 091d1a5f-48be-4a1b-85a9-4a051714b675
transactions 47ac3a8c-3e02-4732-8fa1-02997ba968b2


In [14]:
for endpoint in ['customer', 'transactions']:
    if endpoint == 'customer':
        for i, c in enumerate(API.get(endpoint+'s')):
            if c is None:
                break
            print(endpoint, i, next(API.get(endpoint, {'cid': c})))
    elif endpoint == 'transactions':
        for i, t in enumerate(API.get(endpoint, {'ts': 0, 'te': 5})):
            if t is None:
                break
            print(endpoint, t)

customer 0 christopherlynch@lawson-davis.net
customer 1 nguyenbrenda@yahoo.com
customer 2 hraymond@hotmail.com
customer 3 kimmccarthy@hotmail.com
customer 4 matthewskevin@hotmail.com
transactions 8166176f-4ed9-4b7c-b614-e550733ff0d9
transactions 2fbb6550-fba3-43a7-9c7e-5f380f11dd19
transactions 7211bb90-dcb1-41c6-a1b5-62e0ee6cba0b
transactions d46650c4-51d7-41bc-9ec7-9f1d018ee03d
transactions b8a1de89-00ef-4706-a028-95531ff8c8b8


In [15]:
from IPython.lib.display import YouTubeVideo
YouTubeVideo('8bZh5LMaSmE?t=350')

[https://github.com/emilybache/GildedRose-Refactoring-Kata](https://github.com/emilybache/GildedRose-Refactoring-Kata)

https://github.com/tomquirk/realestate-com-au-api/blob/8368da02a67aaf1c2fe9634f19181fb54685718d/realestate_com_au/realestate_com_au.py#L70-L118

https://softwareengineering.stackexchange.com/questions/351389/dynamic-dispatch-from-a-string-python

In [48]:
from api import API

def request(endpoint, kwargs={}):
    for r in API.get(endpoint, kwargs):
        if r is None:
            break
        yield r


for endpoint in ['customer', 'transactions']:
    if endpoint == 'customer':
        for i, c in enumerate(request(endpoint+'s')):
            print(endpoint, i, next(request(endpoint, {'cid': c})))
    elif endpoint == 'transactions':
        for i, t in enumerate(request(endpoint, {'ts': 0, 'te': 5})):
            print(endpoint, i, t)
endpoint = ''

customer 0 fwiggins@hotmail.com
customer 1 john63@gmail.com
customer 2 joel10@yahoo.com
customer 3 davismaria@gmail.com
customer 4 newmanrichard@hotmail.com
transactions 0 a8a563e6-5fb6-4137-bd6c-55ff8ad3bf8e
transactions 1 afc566a1-8b3c-419c-a4a5-3d74b5f34157
transactions 2 c8af3f9a-55cb-4879-8279-54d99d5c83c0
transactions 3 992b8885-20d6-4f2c-9c4b-03173dd389b7
transactions 4 98e23b01-3602-46f1-9557-9ec0648be198


In [49]:
from dataclasses import dataclass, field
from typing import Generator


# TODO: implement "extra" params e.g. limit/type=='all'
@dataclass
class Endpoint:
    endpoint: str = field(init=False)

    def params(self) -> dict:
        return {}

    def get(self, kwargs) -> Generator:
        yield from request(self.endpoint, self.params(**kwargs))

    
@dataclass
class Transactions(Endpoint):
    endpoint: str = 'transactions'

    def params(self, ts: int, te: int) -> dict:
        return {'ts': ts, 'te': te}

        
@dataclass
class Customer(Endpoint):
    endpoint: str = 'customer'
        
    def params(self, cid) -> dict:
        return {'cid': cid}


@dataclass
class Customers(Endpoint):
    endpoint: str = 'customers'

    def get(self) -> Generator:
        for result in request(self.endpoint, self.params()):
            for cust in Customer().get({'cid': result}):
                yield cust

print(Transactions())

Transactions(endpoint='transactions')


In [50]:
t = Transactions()
print(t)
for i in t.get({'ts':0,'te':5}):
    print(i)

Transactions(endpoint='transactions')
a1f38600-4e70-4114-90ad-64a31f5a4b5a
eae05d5e-a423-4485-90c5-a93617966fcc
ef0c3db6-5a78-402b-b927-64019fff22c1
fe50d5a2-8dfb-4ad3-abc0-a49dd2df24a9
2ebceb26-40ae-42bc-b121-90f0f7e283d5


In [51]:
c = Customers()
print(c)
for i in c.get():
    print(i)

Customers(endpoint='customers')
joshuaodonnell@hotmail.com
robert02@yahoo.com
edwin42@yahoo.com
christopher14@lane.com
williamsmaria@shaffer.com


In [52]:
from dataclasses import dataclass
from typing import Generator

# TODO: implement "extra" params e.g. limit/type=='all'
@dataclass
class Transactions:
    ts: int
    te: int
    endpoint: str = 'transactions'
    
    @property
    def params(self) -> dict:
        return {'ts': self.ts, 'te': self.te}

    def get(self) -> Generator:
        yield from request(self.endpoint, self.params)

        
@dataclass
class Customer:
    cid:      int
    endpoint: str = 'customer'
        
    @property
    def params(self) -> dict:
        return {'id': self.cid}
    
    def get(self):
        yield from request(self.endpoint, self.params)

@dataclass
class Customers:
    endpoint: str = 'customers'
        
    @property
    def params(self) -> dict:
        return {}

    def get(self) -> Generator:
        yield from request(self.endpoint, self.params)
        
print(list(Customers().get()))
print(list(Transactions(ts=0, te=5).get()))


[5646, 8266, 5008, 1245, 5675]
['8ed2fd4a-07d2-4eb6-b039-fbf431e27615', '4fe55cbf-b53c-42c0-a9b9-a9db3e742264', '57b6d668-55c0-43b5-82c2-2b229cb69985', '8e692b65-1b56-4a41-9c7e-90903f989e23', 'db28b6d5-704c-4f53-8280-fd459e6ee3b2']


In [55]:
from dataclasses import dataclass, field
from typing import Generator

from api import API

def request(endpoint, kwargs={}):
    for r in API.get(endpoint, kwargs):
        if r is None:
            break
        yield r

@dataclass
class Stream:
    def params(self) -> dict:
        return {}

    def get(self) -> Generator:
        yield from request(self.endpoint, self.params())

    
@dataclass
class Transactions(Stream):
    ts:       int = ''
    te:       int = ''
    endpoint: str = 'transactions'

    def params(self) -> dict:
        return {'ts': self.ts, 'te': self.te}

        
@dataclass
class Customer(Stream):
    cid:      int = ''
    endpoint: str = 'customer'
        
    def params(self) -> dict:
        return {'cid': self.cid}


@dataclass
class Customers(Stream):
    endpoint: str = 'customers'

    def get(self) -> Generator:
        for result in request(self.endpoint, self.params()):
            yield from Customer(cid=result).get()


STREAMS = {
    'customers':    Customers,
    'transactions': Transactions
}

In [56]:
def run(config):
    for stream, conf in config.items():
        worker = STREAMS[stream](**conf)
        for result in worker.get():
            print(stream, worker, result)

run({
    'customers': {},
    'transactions': {'ts': 0, 'te': 5},
})

customers Customers(endpoint='customers') howardalexandra@gmail.com
customers Customers(endpoint='customers') nchristian@gmail.com
customers Customers(endpoint='customers') christopherfoley@pitts.com
customers Customers(endpoint='customers') aaron76@henry.com
customers Customers(endpoint='customers') julie62@leon.net
transactions Transactions(ts=0, te=5, endpoint='transactions') fd07e6ea-314d-4753-b5f4-9246d53db628
transactions Transactions(ts=0, te=5, endpoint='transactions') af112964-b5d7-4b9d-b4c0-8075b0710524
transactions Transactions(ts=0, te=5, endpoint='transactions') bd540135-383d-4aee-aaca-34065859cbaa
transactions Transactions(ts=0, te=5, endpoint='transactions') 112e88f0-9bc3-4c9d-b8b3-e958792c2392
transactions Transactions(ts=0, te=5, endpoint='transactions') 139d273b-c243-4e05-9bf6-598143eb546d
